### Libraries needed and installed

In [1]:
#pip install pandas
#pip install pandas openpyxl

### Used to Reference

In [2]:
#Categories: 
 #"Batman"
 #"Clothing"
 #"Credit Card Payment"
 #"Food"
 #"Fun"
 #"Health"
 #"Home"
 #"Other"
 #"Rent"
 #"Salary"
 #"Technology"
 #"Transportation"

### Needed to Run 

In [3]:
import pandas as pd
import openpyxl
import os

### Exporting Only Selected Columns Into Excel WB

In [4]:
#List of CSV files to import data from
csv_file_info = [
    {"file_name": "Chase Checkings.csv", "date_column": "Posting Date", "description_column": "Description", "amount_column": "Amount"},    
    {"file_name": "Chase Amazon Credit Card.csv", "date_column": "Transaction Date", "description_column": "Description","amount_column":"Amount"},
    {"file_name": "Discover It.csv", "date_column": "Transaction Date", "description_column": "Description","amount_column":"Amount"},
    {"file_name": "WF Checkings.csv", "date_column": "Transaction Date", "description_column": "Description", "amount_column": "Amount"},
    {"file_name": "WF Platinum.csv", "date_column": "Transaction Date", "description_column": "Description","amount_column":"Amount"},
    {"file_name": "Capital One Platinum.csv", "date_column": "Transaction Date", "description_column": "Description","amount_column":"Amount"},
    {"file_name": "Capital One Savor One.csv", "date_column": "Transaction Date", "description_column": "Description","amount_column":"Amount"},
    {"file_name": "Capital One Venture.csv", "date_column": "Transaction Date", "description_column": "Description","amount_column":"Amount"}
]

#The directory to this python file
script_directory = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()

#Path to the folder containing the CSV files above
folder_path = os.path.join(script_directory, 'To Year Transactions/')

#Path to the existing Excel workbook within the same directory 
excel_file_path = os.path.join(script_directory, 'Budgetbook_Automated.xlsx')

#Run through the list of CSV files and import data with dynamic column selection
for csv_info in csv_file_info:
    csv_file_name = csv_info["file_name"]
    date_column_name = csv_info["date_column"]
    description_column_name = csv_info["description_column"]
    amount_column_name = csv_info["amount_column"]
    
    #Load the existing Excel workbook for each file.
    excel_book = openpyxl.load_workbook(excel_file_path)
    
    #Create an Excel writer object for appending/adding data to the existing workbook.
    with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as excel_writer:
        excel_writer.book = excel_book

        #Indicating which sheet to choose if it exists. If it doesn't exist it creates it.
        if 'Data' in excel_writer.book.sheetnames:
            excel_sheet = excel_writer.book['Data']
        else:
            excel_sheet = excel_writer.book.create_sheet(title='Data')

        
        try:
            #Constructing the complete path to the CSV files.
            csv_file_path = os.path.join(folder_path, csv_file_name)
            
            #Reading the CSV files into a pandas DataFrame.
            df = pd.read_csv(csv_file_path)

            #Creating a new column that refers to the file name of the CSV files
            df['Source'] = csv_file_name
        

            
        #These are the columns being input into the Excel Workbook
            #Extract month and year from 'Transaction Date' and add them as separate columns.
            df['Month'] = pd.to_datetime(df[date_column_name]).dt.strftime('%B')  #%B gives full month name from the 'Transaction Date'
            df['Year'] = pd.to_datetime(df[date_column_name]).dt.year #Gives the year from the 'Transaction Date'
            df['Source'] = df['Source'].str.replace('.csv', '') #This removes the .csv from the file source
            df['Category'] = 'Other'#For the 'Category' column created 'Other' will be the autofilled unless otherwise indicated below
            df['Expected'] = 'No' #For the 'Expected' column created, 'No' will be the autofilled data unless otherwise indicated below
            
#Assigning Categories to Transactions            
            #df.loc[df['Description'].str.contains("Original1|Original2|Original3"), 'Category'] = "Batman"
            df.loc[df['Description'].str.contains("C PAYROLL|SALT LAKE CITY PAYROLL|ZERO MOTORCYCLES DIRECT DEP"), 'Category'] = "Salary"
            df.loc[df['Description'].str.contains("FID BKG|INTERNATIONAL PURCHASE TRANSACTION|Apple Cash TRANSFER|PLATINUM CARD XXXXXXXXXXXX|IRS USATAXPYMT|EVERYDAY CHECKING XXXXXX|DISCOVER E-PAYMENT|APPLE CASH SENT|ONLINE TRANSFER FROM JUNGO J CHECKING|ONLINE TRANSFER TO JUNGO J CHECKING|WELLS FARGO Fee Refund|PURCHASE WITH CASH BACK|ATM WITHDRAWAL AUTHORIZED|ZELLE TO|CHASE CREDIT CRD EPAY|CHASE CREDIT CRD AUTOPAY|INTEREST|APPLECARD GSBANK PAYMENT|CAPITAL ONE MOBILE PMT|CAPITAL ONE CRCARDPMT|ONLINE PAYMENT|Credit One Bank Payment|KLARNA|UPLIFT|AFFIRM|AFTERPAY"), 'Category'] = "Banking"
            df.loc[df['Description'].str.contains("CITY PET CLUB|MA & PAWS|HEALTHY PETS|PetSmart|Feed|Leash"), 'Category'] = "Batman"
            df.loc[df['Description'].str.contains("Ever Forward|Nike"), 'Category'] = "Clothing"
            df.loc[df['Description'].str.contains("SAFEWAY|FOOD MAXX|GROCERY OUTL|COSTCO WHSE|RANCHO MARKETS|SMITHS|SCOTTS VALLEY MARKET|DOMINO'S|POPEYES|MCDONALD'S|SUBWAY|STARBUCKS|IN N OUT|DOORDASH|SUNNY HONEY|CHIPOTLE|REDWOOD SANDWICH|DUTCHBROSLL|KITCHEN"), 'Category'] = "Food"            
            df.loc[df['Description'].str.contains("Cinema|Soundwell|SKY SLC"), 'Category'] = "Fun"
            df.loc[df['Description'].str.contains("VASAFIT|EOS FITNESS|GOLDS GYM|KENKO|Bare Perfo|Gym"), 'Category'] = "Health"
            df.loc[df['Description'].str.contains("REMITLY|GoFundMe|WASH LAUNDRY KIOSK|TARGET|BEST BUY|Wal-Mart|LAUNDRY DAY|ROSS STORE|INTERNATIONAL|INTL"), 'Category'] = "Other"
            df.loc[df['Description'].str.contains("SPOTIFY|GODADDY|AVAST Software|Apple Services|Disney|Fubotv|Microsoft|hostinger"), 'Category'] = "Subscription"
            df.loc[df['Description'].str.contains("PILOT|ARCO|COSTCO GAS|CHEVRON|SHELL|QUIK STOP|MAVERIK|7-ELEVEN|PARKING|LIM*RIDE|Uber|FCP Euro|AUTOZONE|Spin"), 'Category'] = "Transportation"
            
            #Debugging print statements
            print(f"Importing data from file: {csv_file_name}") #Will be printed once the data is imported into the excel workbook one csv file at a time

            #The DataFrame to the 'Data' sheet in the Excel workbook.
            df_filtered = df[[date_column_name, description_column_name, amount_column_name,'Month','Year', 'Source', 'Category','Expected']]  #Selects specific columns
            df_filtered.columns = ['Date', 'Description', 'Amount','Month','Year', 'Source', 'Category','Expected']  #Renames columns for consistency

            #Calculates the row where the new data will be inerted
            if 'Data' in excel_writer.book.sheetnames:
                existing_data = pd.read_excel(excel_file_path, sheet_name='Data')
                start_row = existing_data.shape[0] + 2  #Add 2 for header and 0-based index offset
            else:
                start_row = 1  #Starts at the first row if no data exists yet

            #This is the order for the data frame to be imported - custom how you want it
            for idx, row in df_filtered.iterrows():
                excel_sheet.cell(row=start_row + idx, column=1, value=row['Date'])
                excel_sheet.cell(row=start_row + idx, column=2, value=row['Description'].upper()) #Will convert to uppercase
                excel_sheet.cell(row=start_row + idx, column=3, value=row['Amount'])
                excel_sheet.cell(row=start_row + idx, column=4, value=row['Month'])
                excel_sheet.cell(row=start_row + idx, column=5, value=row['Year'])
                excel_sheet.cell(row=start_row + idx, column=6, value=row['Category'])
                excel_sheet.cell(row=start_row + idx, column=7, value=row['Expected'])
                excel_sheet.cell(row=start_row + idx, column=8, value=row['Source'])

            print(f"Data from file '{csv_file_name}' imported successfully.")
        except Exception as e:
            print(f"Error importing data from file '{csv_file_name}': {str(e)}") #Will be printed if the data is not imported to the excel workbook

print(f"Selected columns from all CSV files have been imported into the 'Data' sheet of '{excel_file_path}'.") #Will be printed once all the data indicated above is imported to the excel workbok


C:\Users\ezequ\AppData\Local\Temp\ipykernel_5644\463391706.py:35: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  excel_writer.book = excel_book
C:\Users\ezequ\AppData\Local\Temp\ipykernel_5644\463391706.py:60: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Source'] = df['Source'].str.replace('.csv', '') #This removes the .csv from the file source


Importing data from file: Chase Checkings.csv
Data from file 'Chase Checkings.csv' imported successfully.


C:\Users\ezequ\AppData\Local\Temp\ipykernel_5644\463391706.py:35: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  excel_writer.book = excel_book
C:\Users\ezequ\AppData\Local\Temp\ipykernel_5644\463391706.py:60: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Source'] = df['Source'].str.replace('.csv', '') #This removes the .csv from the file source


Importing data from file: Chase Amazon Credit Card.csv
Data from file 'Chase Amazon Credit Card.csv' imported successfully.


C:\Users\ezequ\AppData\Local\Temp\ipykernel_5644\463391706.py:35: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  excel_writer.book = excel_book
C:\Users\ezequ\AppData\Local\Temp\ipykernel_5644\463391706.py:60: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Source'] = df['Source'].str.replace('.csv', '') #This removes the .csv from the file source


...

Importing data from file: Capital One Platinum.csv
Data from file 'Capital One Platinum.csv' imported successfully.


C:\Users\ezequ\AppData\Local\Temp\ipykernel_5644\463391706.py:35: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  excel_writer.book = excel_book
C:\Users\ezequ\AppData\Local\Temp\ipykernel_5644\463391706.py:60: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Source'] = df['Source'].str.replace('.csv', '') #This removes the .csv from the file source


Importing data from file: Capital One Savor One.csv
Data from file 'Capital One Savor One.csv' imported successfully.


C:\Users\ezequ\AppData\Local\Temp\ipykernel_5644\463391706.py:35: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  excel_writer.book = excel_book
C:\Users\ezequ\AppData\Local\Temp\ipykernel_5644\463391706.py:60: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Source'] = df['Source'].str.replace('.csv', '') #This removes the .csv from the file source


Importing data from file: Capital One Venture.csv
Data from file 'Capital One Venture.csv' imported successfully.
Selected columns from all CSV files have been imported into the 'Data' sheet of 'C:\Users\ezequ\OneDrive\Documents\Python Scripts\Budgetbook_Automated.xlsx'.
